<a href="https://colab.research.google.com/github/achanhon/coursdeeplearningcolab/blob/master/cnn_cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TP : VGG13 sur CIFAR10

In [ ]:
import torch

def compute_accuracy(batchprovider,net):
  with torch.no_grad():
    net.eval()
    net=net.cuda()
    nb, nbOK = 0, 0
    for x,y in batchprovider:
      x,y = x.cuda(),y.cuda()
      z = net(x)
      
      _,z = z.max(1)
      good = (y==z).float()
      nb+=good.shape[0]
      nbOK+=good.sum().cpu().numpy()
  return nbOK,nb

import random
import collections

def simple_training(batchprovider,net,lr,nbepoch):
  net.train()
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
  meanloss = collections.deque(maxlen=200)
  for epoch in range(nbepoch):
    print(epoch,"/",nbepoch)
    nb, nbOK = 0, 0
    for x,y in batchprovider:
      x,y = x.cuda(),y.cuda()
      z = net(x)

      loss = criterion(z, y)
      meanloss.append(loss.cpu().data.numpy())

      optimizer.zero_grad()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(net.parameters(), 10)
      optimizer.step()

      _,z = z.max(1)
      good = (y==z).float()
      nb+=good.shape[0]
      nbOK+=good.sum().cpu().numpy()

      if random.randint(0, 30) == 0:
        print("average loss=", (sum(meanloss) / len(meanloss)))
    print("average train accuracy",nbOK/nb)
    if nbOK/nb>0.98:
      print("early stopping")
      return

VGG13 et CIFAR10

In [ ]:
import torchvision

print("load model")
net = torchvision.models.vgg13(pretrained=True)
net.avgpool = torch.nn.Identity()
net.classifier = torch.nn.Linear(512, 10)
net = net.cuda()

print("load data")
trainset = torchvision.datasets.CIFAR10(
    root="build",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)
testset = torchvision.datasets.CIFAR10(
    root="build",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=True, num_workers=2
)

print("finetune the model on the data")
simple_training(trainloader,net, 0.0001,8)

print("eval model")
nbok,nb = compute_accuracy(testloader,net)
print("test accuracy",nbok/nb)